<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%206.2%20(2021-02-17).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 6.2 (2021-02-17)**
Week 6: Accelerating towards the final project!

In [ ]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 6-2/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [ ]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)
  fig = plt.figure(figsize=(16,6))
  
  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return plt

## myPlot(): properly formats time domain plot of a signal

In [ ]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [ ]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,6))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')
  return fig, plt  

## myPlotFFTPhase

In [ ]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [ ]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Today's sound file

In [ ]:
inxs_s, fs44 = sf.read(ClassPath + 'INXS-44kHz.wav')
inxs = np.mean(inxs_s,axis=1)
ipd.Audio(inxs,rate=fs44)

# Final Project: Compression

In [ ]:
fs = 44100
bits = 16
channels = 2

bitrate = fs * bits * channels
bitrate

## Quantization

In [ ]:
n_bits = 8
inxs_q = np.floor(inxs * 2**(n_bits-1))
myPlot(inxs_q)
ipd.Audio(inxs_q, rate=fs44)

# Project: Perceptual audio coding

In [ ]:
fs = 22050
f0 = 5000
t = np.arange(fs) / fs
s = np.sin(2*np.pi*f0*t)
ipd.Audio(s,rate=fs)

In [ ]:
f_delta = 5

s_mod = np.sin(2*np.pi*f0*t + 5*np.sin(2*np.pi*f_delta*t))
ipd.Audio(s_mod,rate=fs)

In [ ]:
dur = 5
fs22 = 22050
f1 = 100
f2 = 1500

chirps = []
t = np.arange(fs22 * dur) / fs22
f_up = (f2 - f1)/dur;

chirp = np.sin(np.pi*f_up*t*t + 2*np.pi*f1*t);
ipd.Audio(chirp, rate=fs22)

In [ ]:
f_c = 950
t2 = np.arange(np.ceil(fs22/4)) / fs22
tone = np.sin(2*np.pi * f_c * t2) * np.hanning(len(t2))
tone = np.append(tone, np.zeros(int(fs22/4)))
tones = np.tile(tone,2*dur)
plt.plot(tones)
plt.xlim(0,22050)
ipd.Audio(tones,rate=fs22)

In [ ]:
test = chirp + 0.01*tones
ipd.Audio(test,rate=22050)

In [ ]:
splot = plotSpectrogram(test, 22050, nseg=2048, olap=1920, fft_len=4096)
splot.ylim(0,2000)

In [ ]:
f_p = 1000    # Frequency of peak (masking tone)
f_m = 900     # Frequency of masked tone 
a_m = 0.01    # Amplitde of masked tone

tt = np.arange(fs22)/fs22 # One second of samples

peak_tone = np.sin(2*np.pi * f_p * tt)
mask_tone = a_m * np.sin(2*np.pi * f_m * tt)

test_tone = peak_tone + mask_tone

#fade = np.hanning(2048)
#fade_in = fade[:1024]
#fade_out = fade[1024:2048]

#test_tone[:1024] *= fade_in
#test_tone[len(test_tone)-1024:len(test_tone)] *= fade_out

plt.plot(test_tone)
ipd.Audio(test_tone, rate=fs22)

In [ ]:
sf.write(ClassPath+'sounds/' + username + '-%d-%0.4f.wav' %(f_m, a_m),test_tone,22050)

In [ ]:
myPlotFFT(test_tone / sum(np.abs(test_tone)), fs=22050, x_lim=2000)
#plt.savefig(ClassPath + 'figs/' + username + '-%d-%0.4f.png' %(f_m, a_m))

# Convolution via FFT and inverse FFT...

In [ ]:
f_start = 250000
f_size = 2048
nFFT = 2048
L = 64
win = np.ones(f_size) # np.hanning(f_size)

sig = inxs[f_start:f_start+f_size]
S = np.fft.fft(sig * win, nFFT)
H = np.append(np.ones(L), np.zeros(int(nFFT/2)-L))
H = np.append(H, np.flipud(H))
h = np.real( np.fft.ifft(H) )

S_f = S * H
s_f = np.real( np.fft.ifft((S_f)) )
fig = plt.figure(figsize=(16,6))
plt.subplot(311)
plt.plot(sig * win)
plt.subplot(312)
plt.plot(s_f)
plt.subplot(313)
s_f2 = signal.lfilter(h, 1, sig*win)
plt.plot(s_f2)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(h)

In [ ]:
fig = plt.figure(figsize=(16,6))
plt.subplot(211)
plt.plot(s_f - s_f2)
plt.subplot(212)
plt.plot(np.arange(f_size),s_f, np.arange(f_size),s_f2)
plt.xlim(0,200)

In [ ]:
fig = plt.figure(figsize=(16,6))
s_f3 = signal.lfilter(h, 1, np.append(sig, np.zeros(f_size)))
plt.subplot(211)
plt.plot(s_f3)
plt.subplot(212)
plt.plot(range(f_size*2),np.append(s_f, s_f - s_f2))

In [ ]:
fig = plt.figure(figsize=(16,6))
h2 = np.fft.fftshift(h)
s_f4 = signal.convolve(h2,sig)
plt.subplot(211)
plt.plot(s_f4)
plt.subplot(212)
plt.plot(range(f_size*2),np.append(s_f, s_f - s_f2))

## What's going on... ?